# My first molecular dynamics simulation

# Click 'Edit App' to start coding

In [ ]:
import time
import random

import sys

from io import StringIO
from IPython import get_ipython

from openmm.app import *
from openmm import *
from simtk.unit import *

import openmmtools as tools
import MDAnalysis as md
import nglview as ng
import pandas as pd
import numpy as np

# below prevents warnings from outdated Pandas append command clogging output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt

t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )


In [ ]:
class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        print("Water model undefined")  # optionally print some message to stdout, too
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


In [ ]:
### 1. Choose a water model
h2o = "NONE"
# h2o = "spce"
# h2o = "tip3p"
# h2o = "tip4pew"
# h2o = "tip5p"

if h2o =="NONE":
    raise IpyExit

### 2. Create initial coordinates and system
water = tools.testsystems.WaterBox(model=h2o,
                                   box_edge=2.1*nanometers,
                                   ewaldErrorTolerance=1e-04)

### 2a. Save initial coordinates to file
output = open('water.pdb', 'w')
PDBFile.writeFile(water.topology, water.positions, output)
output.flush()

In [ ]:
### 3. Choose parameters of the experiment: temperature, pressure and length
temperature = 288*kelvin
pressure = 1*atmosphere
simulationTime = 10*picosecond

### 3a. Simulation timestep, total number of steps and thermostat/barostat parameters
timeStep = 0.002*picoseconds
totalSteps = simulationTime / timeStep
frictionCoeff = 0.1/picosecond
nupdt = 20

### 4. Choose an integrator
integrator = LangevinIntegrator( temperature, frictionCoeff, timeStep )
water.system.addForce(MonteCarloBarostat( pressure , temperature , nupdt ) )

### 5 Create a simulation object
simulation = Simulation(water.topology, water.system, integrator)
simulation.context.setPositions(water.positions)
simulation.context. setVelocitiesToTemperature(temperature, random.randrange(99999))

### 6. Set output for trajectory and thermodynamics data
simulation.reporters.append(DCDReporter('traj.dcd', 10))

### 7. Set output for thermodynamics data
simulation.reporters.append(StateDataReporter( "thermo.csv"
                                             , 10
                                             , step            = True
                                             , time            = True
                                             , potentialEnergy = True
                                             , temperature     = True
                                             , density         = True
                                             , volume          = True
                                             , separator       = ","
                                             ))

### 7a. Set screen output to monitor the simulation progress
simulation.reporters.append(StateDataReporter( sys.stdout
                                             , int(totalSteps/20)
                                             , step            = True
                                             , progress        = True
                                             , remainingTime   = True
                                             , elapsedTime     = True
                                             , separator       = " "
                                             , totalSteps      = totalSteps
                                             ))


In [ ]:
### 8. Run MD
simulation.step(totalSteps)

In [ ]:
### 9. Visualization
sys = md.Universe("water.pdb", 'traj.dcd')
view = ng.show_mdanalysis(sys, gui=True)
view.center()
view.representations = [
    {"type": "ball+stick", "params": {"sele": "all"}}
]
view

In [ ]:
### 10. Load the thermodynamic data in a dataframe
data = pd.read_csv("thermo.csv")
# print(data.head())
print(data.tail())

In [ ]:
x = data["Time (ps)"]
y = data["Box Volume (nm^3)"]
plt.plot(x,y)
plt.show()